# **Machine Learning Training**

### Initial Imports:

In [11]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 300)

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

#### Read in CSV as Pandas DataFrame:

In [18]:
# Set path to CSV and read in CSV
csv_path = Path('Returns and Signals.csv')
return_signal_df=pd.read_csv(csv_path)
# Set index as datetime object and drop extraneous columns
return_signal_df.set_index(pd.to_datetime(return_signal_df['Date'], infer_datetime_format=True), inplace=True)
return_signal_df.drop(columns=['Date'], inplace=True)
return_signal_df.head()

,TSLA_Adj Close,TSLA_Volume,TSLA_Returns,vol_trend_signal,bollinger_signal,obv_crossover_signal,sentiment_signal
Date,,,,,,,
2019-01-03,60.071999,34826000,-0.031472,1.0,0.0,-1.0,-1
2019-01-17,69.461998,18383500,0.003641,1.0,0.0,1.0,-1
2019-01-18,60.452000,120754000,-0.129711,-1.0,0.0,-1.0,-1
2019-01-22,59.784000,60333500,-0.011050,-1.0,0.0,-1.0,1
2019-01-23,57.518002,62650000,-0.037903,-1.0,0.0,-1.0,0


#### Check Data Quality:

In [21]:
return_signal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 416 entries, 2019-01-03 to 2020-12-30
Data columns (total 7 columns):
TSLA_Adj Close          416 non-null float64
TSLA_Volume             416 non-null int64
TSLA_Returns            416 non-null float64
vol_trend_signal        416 non-null float64
bollinger_signal        416 non-null float64
obv_crossover_signal    416 non-null float64
sentiment_signal        416 non-null int64
dtypes: float64(5), int64(2)
memory usage: 26.0 KB
